In [48]:
print("hello World!")

hello World!


In [49]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import os
import scipy.stats as st
import numpy as np

In [50]:
# Study data files
project_path = "Project_df.csv"

# Read the csv file
Project_df = pd.read_csv(project_path)

# Display the data table for preview
Project_df

,Client ID,Location,Month,Quarter,Year,Category,Distributor,Total Sales,Total Points,Points Worth ($)
0,2,"Tlalnepantla, Ciudad de Mexico",1,1,2018,C4,D5,51951,0,0.00
1,2,"Tlalnepantla, Ciudad de Mexico",2,1,2018,C4,D5,15301,0,0.00
2,2,"Tlalnepantla, Ciudad de Mexico",5,2,2018,C4,D5,19807,0,0.00
3,2,"Tlalnepantla, Ciudad de Mexico",6,2,2018,C4,D5,16916,0,0.00
4,2,"Tlalnepantla, Ciudad de Mexico",8,3,2018,C4,D5,88353,1080,928.80
...,...,...,...,...,...,...,...,...,...,...
14579,1371,"Colima, Colima",11,4,2020,C4,D12,218149,164,141.04
14580,1371,"Colima, Colima",12,4,2020,C4,D12,119233,10334,8887.24
14581,1372,"Tezontepec, Hidalgo",10,4,2020,C2,D15,26834,172,147.92
14582,1372,"Tezontepec, Hidalgo",11,4,2020,C2,D15,35671,123,105.78


In [51]:
# Sum the Total Sales by Year
yearly_sales_df = (pd.DataFrame(Project_df.groupby('Year')["Total Sales"].sum()).style.format('${0:,.2f}'))
yearly_sales_df

,Total Sales
Year,
2018,"$318,109,469.00"
2019,"$297,477,446.00"
2020,"$276,662,458.00"


In [52]:
# Checking the number of different Locations
unique_locations = Project_df.drop_duplicates("Location")
locations_number = len(unique_locations["Location"])
locations_number

134

In [53]:
# Filtering the data by year to work with comparisons
Project_18_df = Project_df.loc[Project_df.Year == 2018]
Project_19_df = Project_df.loc[Project_df.Year == 2019]
Project_20_df = Project_df.loc[Project_df.Year == 2020]

In [58]:
# Working with 2018 Data by Category Sales
stats_summary_2018_df = (Project_18_df.groupby("Category")["Total Sales"].agg(["count","min","max","mean","median","var","std","sem"]).style.format('${0:,.2f}'))
stats_summary_2018_df

# Averiguar cómo darle formato a una sola columna - por ejemplo, que "count" no sea currency cómo todo lo
# demás, sino un entero

,count,min,max,mean,median,var,std,sem
Category,,,,,,,,
C1,"$1,297.00",$0.00,"$266,897.00","$19,329.37","$15,882.00","$280,700,450.39","$16,754.12",$465.21
C2,"$1,679.00",$0.00,"$883,310.00","$32,854.53","$28,279.00","$1,099,445,958.73","$33,157.89",$809.21
C3,$831.00,$319.00,"$250,064.00","$53,197.03","$48,830.00","$883,855,335.33","$29,729.70","$1,031.31"
C4,$579.00,$0.00,"$714,675.00","$95,441.40","$86,407.00","$3,431,020,736.59","$58,574.92","$2,434.29"
C5,$226.00,"$2,590.00","$471,584.00","$140,004.55","$139,181.50","$4,078,173,565.60","$63,860.58","$4,247.94"
C6,$343.00,$0.00,"$1,817,422.00","$311,277.50","$269,100.00","$38,392,288,815.20","$195,939.50","$10,579.74"


In [47]:
# Creating a Data Frame for analyzing Total Quarterly Sales by Category 
quarter_category_2018_df = (pd.DataFrame(Project_18_df.groupby(['Quarter', 'Category'])["Total Sales"].sum()).style.format('${0:,.2f}'))
quarter_category_2018_df